# MODIS composites and zonal stats for SOC-D 


### The following will download then average temporally through some MODIS rasters to produce the desired 5 yr May inclusive average for the SOC-D project which will be used to attribute ceh land cover polygons. 

### This notebook processes the 16 day 250m product.

It should be noted that the input rasters have already been averaged by NASA/USGS prior to the averaging below.

**Please register with APPears so you can use your username and password in the workflow**

https://lpdaacsvc.cr.usgs.gov/appeears/

**The first part utilises a function I have written to access the APPears API to download the desired data within an AOI geojson.**

**The second stacks and composites rasters then attributes the shapefiles using functions from my own lib geospatial_learn.** 

https://github.com/Ciaran1981/geospatial-learn

The dependencies for this are numerous and it will take a long time to install SO for brevity...

...rather than you having to wait, I have just provided the necessary functions locally, so provided you work in this folder you can import the function. The files are ```appears_down.py``` and ```shape.py```. This reduces our dendencies considerably.

The locally based functions are imported below.

In [ ]:
from src.appears_down import appears_download
from src.shape import zonal_stats
#from geospatial_learn.shape import zonal_stats
import src.raster as rs
# from geospatial_learn import raster as rs
import os
import pandas as pd

### Part 1

Data download. We use a polygon to demarcate the area of interest. To this end we have an aoi geojson ```aoi.geojson``` which is a part of Wales.

We use the APPears API (functionalised for you in the file ```appears_down.py```)

**Remember to register to obtain a user, password you can use with this**


In [ ]:
# the download folder
out_dir = 'modisimg16day'
os.mkdir(out_dir)

# the vars we need for the function
aoi = "aoi.geojson"
user = 'your user name'
password='your password'
start_date="05-01"
end_date="05-31"

Please use...
```python 
appears_download?
```
...to query the docstring of the fuction and the params will be explained. 

Of most importance here are:
```python
product_layer=[('MOD13Q1.006', '_250m_16_days_NDVI')], recurring=True
```
**The products**
The APPears API requires the NASA/USGS code for the product (```'MOD13Q1.006```) which is 250m MODIS data and layer (```'_250m_16_days_NDVI'```) which is the 16-day NDVI. This allows us to download subset of the product and a similar theory can be applied to Landsat bands etc.

**The search**
The parameters below stipulate that we are searching from start to end of May, from 2011-15 and only within the month of May. If the recurring param was switched to False we'd get all the imagery from 2011-05-01 -> 2015-05-01 rather than only within the month itself.

```python 
start_date, end_date, year_range=[2011, 2015], recurring=True

```


Whilst processing a repeat message will appear (pending > processing > done), followed by some joblib outputs.

In [ ]:
appears_download(aoi, user, password, start_date, end_date, 
                 out_dir, year_range=[2011, 2015], product_layer=[('MOD13Q1.006', '_250m_16_days_NDVI')],
                 recurring=True)

### Part 2: Temporal composite and zonal stats

The following will use the downloaded data and meta to sort through the files and weed out some errors associated with this data layer. The 16 day composites naturally straddle monthly boundaries so if we are to stick strictly to May we must remove the April->May images.

To this end we use pandas and filter the meta data associated with the download.

In [ ]:
# use the handy summary csv to subset our data by date as the filenames are 
# not in an easily understood dating format
df = pd.read_csv(os.path.join(out_dir, 'MOD13Q1-006-Statistics.csv'))

df.head()

There are some April images despite our filtering so we reduce to May only with pandas then make a list of file paths to input in processing.

In [ ]:
# change to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# get the may only images
may = df.loc[(df['Date'].dt.month==5)]

# Get the May file paths
fileList = [os.path.join(out_dir, f+'.tif') for f in may["File Name"]]
fileList.sort()

# Would you believe it  - NASA folks have not formatted the files in the csv correctly
# now we replace the incorrect char 
fileList = [f.replace("MOD13Q1_006", "MOD13Q1.006") for f in fileList]

may

### Phew - finally we process rasters to composites, the steps of which are:

1. Stack the layers
2. Warp the raster to OSGB ('EPSG:27700')
3. Calculate the depth-wise stat


In [ ]:
outRas = os.path.join(out_dir, "ModisMay11-15.tif")
rs.stack_ras(fileList, outRas)

# warp & reproj it for later
repro = outRas[:-4]+"espg27700.tif"
rs._quickwarp(outRas, repro, proj='EPSG:27700')

# As above create the output stacks
compRas = os.path.join(out_dir, "ModisMeanMay11-15.tif")

# Finally, average through the bands 
# 2011-15
rs.stat_comp(repro, compRas, bandList=[1], stat='mean')

### Zonal Stats

Lastly we add the zonal stats using an all touching strategy as the polys are fairly small. A subset of the 2015 lcm is included in the repo ```lcm15.gpkg```

Should you wish to only include pixels not touching borders add...

```python
all_touched=False
```

In [ ]:
# Now we add the zonal stat to each polygon in ceh_lcover
inShp = "lcm15.gpkg"

stats = zonal_stats(inShp, compRas, 1, "May2011_15ndvi")


### Result

You should end up with something like the below.

<img src="figs/ndvi.png" style="height:500px">